<a href="https://colab.research.google.com/github/rishubhkhurana/nlp/blob/main/attnmodels/AttentionFromBasicsWithPAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libs

In [69]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchtext
from torchtext.data import Field, BucketIterator
from torchtext.datasets import Multi30k
import spacy, random
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import time, math

## Book Keeping

In [2]:
!python -m spacy download en > /dev/null 2>&1
!python -m spacy download de > /dev/null 2>&1

## Dataset and Dataloders 

In [3]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [4]:
def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

In [5]:
SRC = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True, include_lengths=True)
TRG = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True, include_lengths=True)

In [9]:
trn_data, val_data, test_data = Multi30k.splits(exts=('.en', '.de'), fields =(SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 955kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 274kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 267kB/s]


In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
train_dl, valid_dl, test_dl = BucketIterator.splits((trn_data, val_data, test_data), batch_size = 64, device = device)

In [13]:
SRC.build_vocab(trn_data, min_freq=2)
TRG.build_vocab(trn_data, min_freq=2)

## Model

In [18]:
enc_dims = 512
dec_dims = 512
enc_embed_dims = 256
dec_embed_dims = 256
enc_dropout = 0.5
dec_dropout = 0.5
src_vocab_size = len(SRC.vocab)
trg_vocab_size = len(TRG.vocab)

In [19]:
batch = next(iter(train_dl))

In [75]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_dims, enc_hidden_dims, dec_hidden_dims, dropout = 0.1):
        super().__init__()
        self.embedder = nn.Embedding(vocab_size, embed_dims)
        self.dropout = nn.Dropout(dropout)
        self.rnn = nn.GRU(embed_dims, enc_hidden_dims, bidirectional = True)
        self.bridge = nn.Linear(2*enc_hidden_dims, dec_hidden_dims)
    def forward(self, src, src_lengths):
        # src: [seq len, batch size]
        # srclengths: [batch size]
        embedded = self.dropout(self.embedder(src))
        # src: [seq len, batch size, embed_dims]
        # packed embedding
        packed_outputs = pack_padded_sequence(embedded, src_lengths, enforce_sorted=False)
        # pass through the rnn 
        packed_outputs, hidden = self.rnn(packed_outputs)
        # hidden: [2, batch size, 2*enc_hidden_dims]
        outputs,_ = pad_packed_sequence(packed_outputs)
        # outputs: [src len, batch size, 2*enc_hidden_dims]
        hidden = torch.cat([hidden[-2,:,:], hidden[-1,:,:]], dim = 1)
        hidden = torch.tanh(self.bridge(hidden))
        # hidden: [batch size, dec_hidden_dims]
        return outputs, hidden.unsqueeze(0)

In [76]:
class Attention(nn.Module):

    def __init__(self, enc_hidden_dims, dec_hidden_dims):
        super().__init__()
        self.energymodel = nn.Linear(2*enc_hidden_dims+dec_hidden_dims, dec_hidden_dims)
        self.valuemodel = nn.Linear(dec_hidden_dims, 1, bias = False)

    def forward(self, hidden, enc_outputs, mask):
        # hidden: [1, batch size, dec_hidden_dims]
        # enc_outputs: [src len, batch size, 2*enc_hidden_dims]
        srclen = enc_outputs.shape[0]
        hidden = hidden.permute((1,0,2)).repeat(1, srclen, 1)
        # hidden: [batch size, seq len, dec_hidden_dims]
        energy = torch.cat([enc_outputs.permute((1,0,2)), hidden], dim = 2)
        energy = torch.tanh(self.energymodel(energy))
        # energy: [batch size, seqlen, dec_hidden_dims]
        values = self.valuemodel(energy).squeeze(-1)
        # values : [batch size, seqlen]
        values = values.masked_fill(mask, -1e10)
        return F.softmax(values, dim = 1)

In [77]:
class Decoder(nn.Module):

    def __init__(self, vocab_size, embed_dims, enc_hidden_dims, dec_hidden_dims, attention, dropout = 0.1):
        super().__init__()
        self.hidden_dims = dec_hidden_dims
        self.output_dims = vocab_size
        self.embedder = nn.Embedding(vocab_size, embed_dims)
        self.dropout = nn.Dropout(dropout)
        self.rnn = nn.GRU(2*enc_hidden_dims + embed_dims, dec_hidden_dims)
        self.attn = attention
        self.classifier = nn.Linear(2*enc_hidden_dims + dec_hidden_dims + embed_dims, vocab_size)

    def forward(self, dec_inp, hidden, enc_outputs, mask):
        # hidden: [1, batch size, dec_hidden_dims]
        # enc_outputs: [src len, batch size, 2*enc_hidden_dims]
        # dec_inp: [batch size]
        dec_inp = dec_inp.unsqueeze(0)
        # dec_inp: [1, batch size]
        embedded = self.dropout(self.embedder(dec_inp))
        # embedded: [1, batch size, embed_dims]
        weighted_attns = self.attn(hidden, enc_outputs, mask)
        # weighted_attns: [batch size, src len]
        weighted_attns = weighted_attns.unsqueeze(1)
        weighted_enc_outputs = torch.bmm(weighted_attns, enc_outputs.permute((1,0,2)))
        # weighted_enc_outputs: [batch size, 1, 2*enc_hidden_dims]
        weighted_enc_outputs = weighted_enc_outputs.permute((1,0,2))
        # weighted_enc_outputs: [1, batch size, 2*enc_hidden_dims]
        outputs, hidden = self.rnn(torch.cat([weighted_enc_outputs, embedded], dim = 2), hidden)
        # outputs: [1, batch size, ], hidden: [1, batch size, dec_hidden_dims]
        classifier_outputs = self.classifier(torch.cat([hidden.squeeze(0), embedded.squeeze(0), weighted_enc_outputs.squeeze(0)], dim = 1))
        
        return classifier_outputs, hidden


In [78]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, pad_idx, device = 'cuda'):
        super().__init__()
        self.pad_idx = pad_idx
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    def create_mask(self, src):
        return (src.permute((1,0))==self.pad_idx)
    def forward(self, src, trg, src_lengths, tforce_ratio = 0.5):
        # src: [srclen, batch size]
        # trg: [trglen, batch size]
        # src_lengths: [batch size]
        trglen = trg.shape[0]
        bs = trg.shape[1]
        vocab_size = self.decoder.output_dims
        outputs = torch.zeros((trglen, bs, vocab_size)).to(self.device)
        enc_outputs, hidden = self.encoder(src, src_lengths)
        dec_inp = trg[0]
        mask = self.create_mask(src)
        # mask: [batch size, srclen]
        for t in range(1, trglen):
            dec_output, dec_hidden = self.decoder(dec_inp, hidden, enc_outputs, mask)
            if random.random()< tforce_ratio:
                dec_inp = trg[t]
            else:
                dec_inp = dec_output.argmax(dim=1)
            outputs[t] = dec_output
        return outputs

## Training Utils

In [79]:
def init_weights(model):
    for name, param in model.named_parameters():
        if name=='weight':
            nn.init.normal_(param.data, mean=0, std=0.01)
        elif name=='bias':
            nn.init.constant_(param.data, 0)


In [80]:
def count_parameters(model):
    total_params=0
    for param in model.parameters():
        total_params+=param.numel()
    return total_params

In [81]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src[0]
        trg = batch.trg[0]
        srclengths = batch.src[1].to('cpu')
        optimizer.zero_grad()
        
        output = model(src, trg, srclengths)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [82]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src[0]
            trg = batch.trg[0]
            srclengths = batch.src[1].to('cpu')

            output = model(src, trg, srclengths, tforce_ratio=0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [83]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Training Model 

In [84]:
enc_dims = 512
dec_dims = 512
enc_embed_dims = 256
dec_embed_dims = 256
enc_dropout = 0.5
dec_dropout = 0.5
src_vocab_size = len(SRC.vocab)
trg_vocab_size = len(TRG.vocab)

In [85]:
pad_idx = SRC.vocab.stoi[SRC.pad_token]

In [86]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [87]:
enc_model = Encoder(vocab_size=src_vocab_size, enc_hidden_dims=enc_dims, 
                    embed_dims = enc_embed_dims, dec_hidden_dims = dec_dims, dropout = enc_dropout).to(device)
attn_model = Attention(enc_hidden_dims=enc_dims, dec_hidden_dims=dec_dims).to(device)
dec_model = Decoder(vocab_size = trg_vocab_size, embed_dims = dec_embed_dims, dec_hidden_dims = dec_dims, 
                    enc_hidden_dims = enc_dims, dropout = dec_dropout, attention = attn_model).to(device)
model = Seq2Seq(encoder = enc_model, decoder = dec_model, device = device, pad_idx = pad_idx).to(device)

In [88]:
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedder): Embedding(5893, 256)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): GRU(256, 512, bidirectional=True)
    (bridge): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (embedder): Embedding(7855, 256)
    (dropout): Dropout(p=0.5, inplace=False)
    (rnn): GRU(1280, 512)
    (attn): Attention(
      (energymodel): Linear(in_features=1536, out_features=512, bias=True)
      (valuemodel): Linear(in_features=512, out_features=1, bias=False)
    )
    (classifier): Linear(in_features=1792, out_features=7855, bias=True)
  )
)

In [89]:
print(f"Total Model parameters: {count_parameters(model):,}")

Total Model parameters: 24,036,783


In [90]:
opt = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [ ]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_dl, opt, criterion, CLIP)
    valid_loss = evaluate(model, valid_dl, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model_type1.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 42s
	Train Loss: 4.844 | Train PPL: 126.956
	 Val. Loss: 4.571 |  Val. PPL:  96.667
Epoch: 02 | Time: 1m 42s
	Train Loss: 3.816 | Train PPL:  45.406
	 Val. Loss: 4.003 |  Val. PPL:  54.789
Epoch: 03 | Time: 1m 42s
	Train Loss: 3.242 | Train PPL:  25.595
	 Val. Loss: 3.791 |  Val. PPL:  44.318
Epoch: 04 | Time: 1m 42s
	Train Loss: 2.836 | Train PPL:  17.041
	 Val. Loss: 3.754 |  Val. PPL:  42.680
Epoch: 05 | Time: 1m 42s
	Train Loss: 2.515 | Train PPL:  12.367
	 Val. Loss: 3.726 |  Val. PPL:  41.497
Epoch: 06 | Time: 1m 42s
	Train Loss: 2.317 | Train PPL:  10.145
	 Val. Loss: 3.765 |  Val. PPL:  43.143
Epoch: 07 | Time: 1m 42s
	Train Loss: 2.229 | Train PPL:   9.291
	 Val. Loss: 3.740 |  Val. PPL:  42.115
Epoch: 08 | Time: 1m 42s
	Train Loss: 2.108 | Train PPL:   8.229
	 Val. Loss: 3.780 |  Val. PPL:  43.818
Epoch: 09 | Time: 1m 42s
	Train Loss: 2.016 | Train PPL:   7.508
	 Val. Loss: 3.822 |  Val. PPL:  45.710
Epoch: 10 | Time: 1m 42s
	Train Loss: 1.946 | Train PPL